In [1]:
from __future__ import division
import tensor_calculation as t
import tensor_advanced as ta
from collections import Counter
import random
from math import exp

In [2]:
def logist(list_):
    if type(list_) != list:
        return (1/(1+exp(-1*list_)))
    return [logist(i) for i in list_ ]     

In [3]:
def de_logist(list_):
    return t.list_muti(logist(list_), t.list_shift(t.list_nu(logist(list_), -1) , 1))

In [4]:
def tanh(list_):
    if type(list_) != list:
        return (1-exp(-2*list_/100))/(1+exp(-2*list_/100))
    return [tanh(i) for i in list_ ] 

In [5]:
def de_tanh(list_):
    inter = t.list_power(tanh(list_), 2)
    return t.list_shift(t.list_nu(inter, -1) ,1)

In [6]:
t.list_shift([1,2,3],-1)

[0, 1, 2]

In [7]:
def relu(list_):
    if type(list_) != list:
        return max(list_,0 )
    return [relu(i) for i in list_ ] 

In [8]:
relu([[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]])

[[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]]

In [9]:
def zero_returrn1(list_):
    if type(list_) != list:
        if list_>0:
            return list_
        return 1
    return [zero_returrn1(i) for i in list_ ] 

In [10]:
zero_returrn1([[1,0],[-1,-2]])

[[1, 1], [1, 1]]

In [11]:
def de_relu(list_):
    if type(list_) != list:
        if list_>0:
            return 1
        elif list_==0:
            return 1/2
        else:
            return 0
    return [de_relu(i) for i in list_ ] 

In [12]:
de_relu([[[[1, 2], [0, 4]], [[5, 6], [7, 8]]], [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]])

[[[[1, 1], [0.5, 1]], [[1, 1], [1, 1]]], [[[1, 1], [1, 1]], [[1, 1], [1, 1]]]]

In [115]:
def dnn (x , w, b, func, softmax):
    if len(w) != len(b) or len(x) !=len(w[0][0]):
        return "wrong parameter"
    o_tensor=[x]
    i_tensor=[]
    for i in range(len(w)-1):
        i_tensor.append(t.list_add(t.tensor_contrac([o_tensor[i]], w[i], [1], [1]), [b[i]])[0] )
        o_tensor.append( apply (func, [i_tensor[i]] ))
    i_tensor.append(t.list_add(t.tensor_contrac([o_tensor[len(w)-1]], w[len(w)-1], [1], [1]), [b[len(w)-1]])[0] )
    if softmax == False:
        o_tensor.append( apply (func, [i_tensor[len(w)-1]] ))
    else:
        o_tensor.append(ta.softmax(i_tensor[len(w)-1]))
    return [o_tensor, i_tensor]

In [95]:
www=ta.gene_random_tensor([2, 2, 2] ,(0,1))+ta.gene_random_tensor([1, 1, 2] ,(0,1))
bbb=ta.gene_random_tensor([2, 2] ,(0,1) )+ ta.gene_random_tensor([1, 1] ,(0,1) )

In [112]:
dnn ([1,0] ,www, bbb, relu, True)

[[[1, 0],
  [0.8113073637479205, 0.6040850775503703],
  [0.9629760035580566, 1.7693379108720735],
  [1.0]],
 [[0.8113073637479205, 0.6040850775503703],
  [0.9629760035580566, 1.7693379108720735],
  [1.5244268505394183]]]

In [128]:
def back_pro(der_e_out, in_tensor, w, de_func, softmax):
    der_o_i= apply(de_func, [in_tensor] )
    der_e_o=[]
    if softmax == False:
        der_e_i=[t.list_muti(der_e_out, der_o_i[len(der_o_i)-1]) ]
    else:
        der_e_i = [ der_e_out]
    for j in range(len(w)-1):
        der_e_o.append(ta.mat_dot([der_e_i[j]], w[len(w)-j-1]) [0])
        der_e_i.append(t.list_muti(der_e_o[j-1], der_o_i[len(der_o_i)-j-2])  )   #start from the second layer     
    #print der_e_o[j-1], der_o_i[len(der_o_i)-j-2]
    return [der_e_i[len(der_e_i)-k-1 ]   for k in  range(len(der_e_i))]

In [17]:
def train_dnn_adp(data, k, width, func, der_func, step, softmax):
    w =ta.gene_random_tensor([1, len(data[0][0] ), width], (-1,1))+ ta.gene_random_tensor([k, width, width], (-1,1))+\
           ta.gene_random_tensor([1, width,  len(data[0][1] ),], (-1,1))
    b =ta.gene_random_tensor([k+1, width], (-1,1))+ ta.gene_random_tensor([1, len(data[0][1] )], (-1,1))
    w_grad_sum=ta.gene_blank_tensor([1, len(data[0][0] ), width] )+ta.gene_blank_tensor([k, width, width] )+\
                           ta.gene_blank_tensor([1, width,  len(data[0][1] )] )
    b_grad_sum=ta.gene_blank_tensor([k+1, width] )+ta.gene_blank_tensor( [1, len(data[0][1] )])
    while True:
        time =0
        for d in data:
            y = dnn(d[0] ,w, b, func, softmax)
            der_e_out =t.list_add(y[len(y)-1], t.list_nu( d[1],-1) )
            der_e_a=back_pro(der_e_out, y, w, der_func, softmax)
            der_e_b=der_e_a
            der_e_w=[ta.mat_dot(ta.colu_fi(y[i] ), [der_e_a[i] ])  for i in range(len(der_e_a))]
            w_grad_sum = t.list_power(t.list_add( t.list_power(w_grad_sum, 2) , t.list_power(der_e_w, 2) ), 1/2)  
            b_grad_sum = t.list_power(t.list_add( t.list_power(b_grad_sum, 2) , t.list_power(der_e_b, 2) ), 1/2) 
            w = t.list_add(w, t.list_muti( t.list_nu(der_e_w, -step) , t.list_power( zero_returrn1(w_grad_sum )  , -1) ))
            b = t.list_add(b, t.list_muti( t.list_nu(der_e_b, -step) , t.list_power( zero_returrn1(b_grad_sum )  , -1) ))
        if t.list_max(der_e_b+ der_e_w) <0.1:
            break
    return [w, b]      

In [168]:
def train_dnn(data, lenth, width, func, der_func, step, softmax):
    if lenth<2:
            w =ta.gene_random_tensor([1, len(data[0][1] ), len(data[0][0] )], (0,1))
            b =ta.gene_random_tensor([1, len(data[0][1] )], (0.1,0.1))
    else:
        w =ta.gene_random_tensor([1, width, len(data[0][0] )], (0,1))+\
             ta.gene_random_tensor([lenth-2, width, width], (0,1))\
            +ta.gene_random_tensor([1,  len(data[0][1] ), width], (0,1))
        b =ta.gene_random_tensor([lenth-1, width], (0.1,0.1))+ ta.gene_random_tensor([1, len(data[0][1] )],(0.1,0.1))
    mini = 50
    n=1
    while True:
        error= []
        batch = random.sample(data, 2)
        sum_e_w=[]
        sum_e_b=[]
        for d in batch:
            out_tensor = dnn(d[0] ,w, b, func, softmax)[0]
            in_tensor = dnn(d[0] ,w, b, func, softmax)[1]
            der_e_out =t.list_add(out_tensor[-1:][0] , t.list_nu(d[1],-1) )
            print out_tensor[-1:][0], d[1], n
            error.append(ta.kl_div(d[1], ta.softmax(out_tensor[-1:][0])))
            der_e_i=back_pro(der_e_out, in_tensor, w, der_func, softmax)
            sum_e_b.append(der_e_i)
            sum_e_w.append([ta.mat_dot(ta.colu_fi(der_e_i[i]  ), [out_tensor[i] ])  for i in range(len(der_e_i))])
        wder_e_w=t.list_sum(sum_e_w)
        wder_e_b=t.list_sum(sum_e_b)
        w = t.list_add(w, t.list_nu(wder_e_w, -1*step/(n)**(1/2)))
        b = t.list_add(b, t.list_nu(wder_e_b, -1*step/(n)**(1/2)))
        n +=1
        if mini >sum(error):
            mini = sum(error)
        #print mini
        if  n==3000: #or mini<0.3:
            break
    return [w, b]      

In [76]:
data_x=ta.gene_random_tensor([100,2], (-1, 1))
data1 = [[x, [1] ]  if x[0]**2+ x[1]**2<0.6**2 else [x, [0] ] for x in data_x ]

In [ ]:
data_n=ta.gene_random_tensor([100,2,1], (-1, 1)), [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]], [[1/2,1/2],[1,0]]

In [150]:
data=[ [[0,1],[0,1]], [[1,1],[1,0]]]

In [71]:
for i in range(10):
    data.append([ta.gene_random_tensor([1,2],(-1,1))[0], ta.gene_random_tensor([1,2],(-1,1))[0]])

In [170]:
train_dnn(data, 1, 2, tanh, de_tanh, 0.5, False)

[0.014884086530907158, 0.011808620608145373] [1, 0] 1
[0.006193637102578948, 0.005529331581217064] [0, 1] 1
[0.015979398781025236, 0.015354513996077497] [0, 1] 2
[0.019507004390791075, 0.01213331771567435] [0, 1] 2
[0.015797540709365208, 0.025802967553920732] [0, 1] 3
[0.029337450373414165, 0.035506632409712593] [1, 0] 3
[0.021303746357311186, 0.031214597622047573] [0, 1] 4
[0.037635851321651405, 0.040812557963822216] [1, 0] 4
[0.029611932380153316, 0.027413247683206198] [0, 1] 5
[0.044724816420650834, 0.045337761163119404] [1, 0] 5
[0.012936333043536979, 0.01998078623669387] [1, 0] 6
[0.033739043914119506, 0.031553571243626996] [0, 1] 6
[0.03561370227732191, 0.03545813817416743] [0, 1] 7
[0.014881662966322706, 0.02191403244236921] [1, 0] 7
[0.03392966936412996, 0.041520360385661786] [0, 1] 8
[0.05456723266679725, 0.05685884131052416] [1, 0] 8
[0.05943053480776438, 0.05993065886692004] [1, 0] 9
[0.03713844515639909, 0.04469696226001851] [0, 1] 9
[0.019780933846928722, 0.026770994972701

[[[[11.579050275684022, 11.73169867667605],
   [12.37398175749143, 12.924410733179942]]],
 [[26.321476172014034, 27.344463776294205]]]

In [104]:
t.tensor_contrac([[0,1]],[[1,2],[2,3],[1,3]],[1],[1])

[[2, 3, 3]]

In [141]:
[rew, reb] =[[[[0.021235445375882372, -0.06929516791746065],
   [0.08171622055850197, -0.21196011658237773],
   [-0.059972042357289104, -0.0371113491178493]],
  [[-0.034271106432513965, -0.235318853566085, 0.019859193528246263],
   [0.024190175848686435, -0.03910900875969712, -0.018504224250929416]]],
 [[-0.09780221610547313, 0.13030122624554613, -5.905132191707469e-07],
  [0.26732803235780705, -0.09808162275157642]]]

In [143]:
dnn ([0,1] , rew, reb, relu, False)[0]

[[0, 1], [0, 0, 0], [0.26732803235780705, 0]]

In [ ]:
t.list_add(ta.mat_dot([[100,-10]], [[-8.541277070379497, -0.03444914524998226],
   [-0.4534548466897445, 0.0682211562226301]]),[[1.1412426073963284, -0.43060873140748956]] )

In [ ]:
tanh(-848.4519159636559)

In [171]:
ta.gene_random_tensor([2, 2], (0.1,0.1))

[[0.1, 0.1], [0.1, 0.1]]